<a href="https://colab.research.google.com/github/rehanbchinoy/Math-156-project/blob/main/odyssey_remake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [ ]:
odyssey = tf.keras.utils.get_file('odyssey.txt', 'http://classics.mit.edu/Homer/odyssey.mb.txt')

 417792/Unknown - 0s 0us/step

### Read the data

First, look in the text:

In [ ]:
# Read, then decode for py2 compat.
text = open(odyssey, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 611300 characters


In [ ]:
# Take a look at the first few hundered characters in text
print(text[10000:10750]) # starts at 260

 will
not be away much longer; for he is a man of such resource that even
though he were in chains of iron he would find some means of getting
home again. But tell me, and tell me true, can Ulysses really have
such a fine looking fellow for a son? You are indeed wonderfully like
him about the head and eyes, for we were close friends before he set
sail for Troy where the flower of all the Argives went also. Since
that time we have never either of us seen the other." 

"My mother," answered Telemachus, tells me I am son to Ulysses, but
it is a wise child that knows his own father. Would that I were son
to one who had grown old upon his own estates, for, since you ask
me, there is no more ill-starred man under heaven than he who they
tell me i


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

74 unique characters


### Vectorize the text

In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Create training examples and targets

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(611300,), dtype=int64, numpy=array([36, 66, 63, ..., 27, 10,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

P
r
o
v
i
d
e
d
 
b


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'P' b'r' b'o' b'v' b'i' b'd' b'e' b'd' b' ' b'b' b'y' b' ' b'T' b'h'
 b'e' b' ' b'I' b'n' b't' b'e' b'r' b'n' b'e' b't' b' ' b'C' b'l' b'a'
 b's' b's' b'i' b'c' b's' b' ' b'A' b'r' b'c' b'h' b'i' b'v' b'e' b'.'
 b'\n' b'S' b'e' b'e' b' ' b'b' b'o' b't' b't' b'o' b'm' b' ' b'f' b'o'
 b'r' b' ' b'c' b'o' b'p' b'y' b'r' b'i' b'g' b'h' b't' b'.' b' ' b'A'
 b'v' b'a' b'i' b'l' b'a' b'b' b'l' b'e' b' ' b'o' b'n' b'l' b'i' b'n'
 b'e' b' ' b'a' b't' b'\n' b' ' b' ' b' ' b' ' b'h' b't' b't' b'p' b':'
 b'/' b'/' b'c'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Provided by The Internet Classics Archive.\nSee bottom for copyright. Available online at\n    http://c'
b'lassics.mit.edu//Homer/odyssey.html\n\nThe Odyssey\nBy Homer\n\n\nTranslated by Samuel Butler\n\n------------'
b'----------------------------------------------------------\n\nBOOK I\n\nTell me, O muse, of that ingeniou'
b's hero who travelled far and wide\nafter he had sacked the famous town of Troy. Many cities did he vis'
b'it,\nand many were the nations with whose manners and customs he was acquainted;\nmoreover he suffered '


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Provided by The Internet Classics Archive.\nSee bottom for copyright. Available online at\n    http://'
Target: b'rovided by The Internet Classics Archive.\nSee bottom for copyright. Available online at\n    http://c'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [ ]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## Try the model


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 75) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  19200     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  76875     
                                                                 
Total params: 4,034,379
Trainable params: 4,034,379
Non-trainable params: 0
_________________________________________________________________


### Attach an optimizer, and a loss function

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 75)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.3162036


In [ ]:
tf.exp(mean_loss).numpy()

74.90372

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

In [ ]:
EPOCHS = 20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
94/94 [==============================] - 20s 186ms/step - loss: 2.8621
Epoch 2/20
94/94 [==============================] - 18s 186ms/step - loss: 2.1454
Epoch 3/20
94/94 [==============================] - 18s 187ms/step - loss: 1.8908
Epoch 4/20
94/94 [==============================] - 18s 184ms/step - loss: 1.6878
Epoch 5/20
94/94 [==============================] - 18s 184ms/step - loss: 1.5374
Epoch 6/20
94/94 [==============================] - 18s 184ms/step - loss: 1.4290
Epoch 7/20
94/94 [==============================] - 18s 184ms/step - loss: 1.3454
Epoch 8/20
94/94 [==============================] - 18s 184ms/step - loss: 1.2788
Epoch 9/20
94/94 [==============================] - 18s 184ms/step - loss: 1.2233
Epoch 10/20
94/94 [==============================] - 18s 184ms/step - loss: 1.1715
Epoch 11/20
94/94 [==============================] - 18s 184ms/step - loss: 1.1235
Epoch 12/20
94/94 [==============================] - 18s 184ms/step - loss: 1.0770
Epoch 13/20
9

## Generate text

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Telemachus'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Telemachus, "Stranger," said Ulysses,
"dy quiver there is nothing better of arrows. Everything as she took his head and bronze
with his old day." 

Minerva answered, "Stranger, do you or night well, and the sure Archare
begins whoever him by force, and saying in his crew while death
will happen as ever you may, your people were
all sorts of going to put his hands over me against I inswered, 'Sir, with the arrows on ours, for some of your
wife and chew brives nord that some of Neptune had had heant of
wien. When they raised a cred town and wealth, for we had had their hosts such as
go to the house there nwill of you to do, from what nurse that she went about getting it,
littening to be well disposed towards the time. My noble savoder
Eubaechan about the young home will not retine stones, and I had had enough
to kill him, instead of a handsome
resounded with places for for Pylos never dinner ready, lunger on
it and she stowed on ining himself in his own eyes, and every
man called me, on 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Odysseus:', 'Odysseus:', 'Odysseus:', 'Odysseus:', 'Odysseus:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'Odysseus: They say you are the sacrefts\nyou will be not marriage in the lot and unborning and\nrauged it. At the hand of heaven was and given me all blood they went to\nthe linen and divided it ready, Telemachus is word on and on with the ever-living\nof king Prelapentane;\nbut when Jove bring them does. \n\n"Even so, then, when I saw him I to return. He snatched up three upon men and smoke Minoss valiant who about my hands\nin Laertes and the sails, took Telemachus to go back and smote the doorway\nof the spoil and wore themselves to make him in the likeness of a wound.\nStick they did not desire. Suppose I must come with my ships against your own people, who are shaft killed him." \n\nThey daughter of Overing-pook were golden and attented them. The Nestor was away\nalded, and every man had entered their mouths trusting with which they sun he touch them as\nthough he were us old and earth. We must not be\nhis feasting to your own cosseate. For the storm wrong the Whole 

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model.save_weights('/content/drive/MyDrive/Colab Notebooks/checkpoint_epoch20')

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/odyssey_saved_model_epoch20')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/odyssey_saved_model_epoch20/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/odyssey_saved_model_epoch20/assets
